
# 📘 Shadowing Pose Trainer & Runtime (OpenCV + MediaPipe)
**Mục tiêu**: 
1) Trích xuất keypoints từ video (hoặc webcam) và **huấn luyện** chuỗi **các pha A→B→C→D→E** cho mỗi động tác trong dataset.  
2) **Suy luận real-time**: hiển thị khung xương (skeleton), tô **xanh** (đúng) / **đỏ** (sai), kèm các **điểm A–E** để người dùng **chạm/click** theo **trình tự** (shadowing).  
3) Xác định **điểm bắt đầu** / **kết thúc** của một **chuỗi động tác** (ví dụ *serve*) và **xuất JSON** để dùng lại.

> 💡 Notebook này đã **chia nhỏ theo cell**, kèm **chú thích chi tiết**, dễ tuỳ biến và bảo trì.


In [3]:

# ==============================
# 1) CÀI ĐẶT THƯ VIỆN (nếu cần)
# ==============================
# 👉 CHẠY cell này trên máy bạn nếu thiếu thư viện.
# Lưu ý: Google Colab / local máy có Internet thì mới pip install được.

# !pip install --upgrade pip
# !pip install opencv-python mediapipe numpy scikit-learn pyyaml
# Tuỳ chọn: dùng hmmlearn cho HMM (nếu muốn)
# !pip install hmmlearn

#print("Nếu đã cài đủ, bạn có thể bỏ qua cell này.")


In [4]:

# ==============================
# 2) IMPORTS & CẤU HÌNH CHUNG
# ==============================
import os, sys, json, time, math, yaml
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional
from pathlib import Path

import numpy as np
import cv2

# Thư viện keypoints
try:
    import mediapipe as mp
except Exception as e:
    mp = None
    print("mediapipe chưa được cài. Hãy pip install mediapipe trước khi chạy trích xuất/real-time.")

# Học máy
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Nếu có hmmlearn thì dùng HMM; nếu không, pipeline sẽ fallback sang KMeans
try:
    from hmmlearn.hmm import GaussianHMM
    HAVE_HMM = True
except Exception:
    HAVE_HMM = False
    print("hmmlearn không có sẵn. Sẽ dùng KMeans thay thế để phân pha A→E.")

# ---- Cấu hình ----
ROOT = Path.cwd()               # Thư mục làm việc
DATA_DIR = ROOT / r"E:\Study\usth\group_project\model\pose_videos" # /<action>/<video files>
FEAT_DIR = ROOT / "features"    # Nơi lưu .npz keypoints & features
MODEL_DIR = ROOT / "models"     # Nơi lưu template A-E per action (JSON)
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(FEAT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# DS hành động. Ví dụ: ["serve", "forehand", "backhand"]
ACTIONS = ["serve", "driveforehand", "drivebackhand"]

# Số pha chuẩn hoá cho mỗi động tác
NUM_PHASES = 5  # A,B,C,D,E

# Khung xương pose của MediaPipe (33 landmarks)
MP_LANDMARK_NUM = 33

# Cặp xương để vẽ (một số cặp tiêu biểu)
SKELETON_EDGES = [
    (11,13),(13,15),  # tay trái
    (12,14),(14,16),  # tay phải
    (11,12),          # vai
    (23,24),          # hông
    (11,23),(12,24),  # thân
    (23,25),(25,27),  # chân trái
    (24,26),(26,28)   # chân phải
]

# Ngưỡng đúng/sai mặc định (cosine/chuẩn hoá) – sẽ hiệu chỉnh theo action sau khi train
DEFAULT_TOL = 0.15

print("Cấu hình xong. DATA_DIR:", DATA_DIR)


hmmlearn không có sẵn. Sẽ dùng KMeans thay thế để phân pha A→E.
Cấu hình xong. DATA_DIR: E:\Study\usth\group_project\model\pose_videos


In [5]:

# =====================================
# 3) TIỆN ÍCH TOÁN HỌC & TÍNH GÓC KHỚP
# =====================================
def unit_vector(v: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(v) + 1e-8
    return v / n

def angle_3pts(a: np.ndarray, b: np.ndarray, c: np.ndarray) -> float:
    """Tính góc ABC (độ) với b là đỉnh, dựa trên 3 điểm 2D/3D."""
    ab = a - b
    cb = c - b
    uab, ucb = unit_vector(ab), unit_vector(cb)
    cosang = np.clip(np.dot(uab, ucb), -1.0, 1.0)
    return float(np.degrees(np.arccos(cosang)))

# Bộ góc tiêu biểu từ Mediapipe (dùng ID joints)
# Có thể tuỳ biến bổ sung
ANGLE_TRIPLETS = [
    (11, 13, 15), # vai-tren tay-tren cổ tay trái
    (12, 14, 16), # vai-tren tay-tren cổ tay phải
    (23, 25, 27), # hông-gối-cổ chân trái
    (24, 26, 28), # hông-gối-cổ chân phải
    (11, 23, 25), # vai trái - hông trái - gối trái
    (12, 24, 26), # vai phải - hông phải - gối phải
]

def normalize_landmarks(landmarks: np.ndarray) -> np.ndarray:
    """Chuẩn hoá 33x(2 or 3) landmarks về hệ toạ độ tương đối:
    - Tịnh tiến theo hông trung bình (pelvis: mean(23,24))
    - Chia tỉ lệ theo khoảng cách vai (11,12)
    """
    assert landmarks.ndim == 2
    assert landmarks.shape[0] == MP_LANDMARK_NUM
    pelvis = (landmarks[23] + landmarks[24]) / 2.0
    shoulders = (landmarks[11], landmarks[12])
    scale = np.linalg.norm(shoulders[0] - shoulders[1]) + 1e-8

    lm = landmarks - pelvis
    lm = lm / scale
    return lm

def landmarks_to_feature(landmarks: np.ndarray) -> np.ndarray:
    """Chuyển landmarks (33x2) -> vector đặc trưng 1D:
    - Tọa độ (x,y) đã chuẩn hoá (66 chiều)
    - Góc khớp (len(ANGLE_TRIPLETS)) chiều
    """
    lm_norm = normalize_landmarks(landmarks[:, :2])
    feats = lm_norm.flatten()  # 66
    # Góc
    angles = []
    for (i,j,k) in ANGLE_TRIPLETS:
        a, b, c = lm_norm[i], lm_norm[j], lm_norm[k]
        angles.append(angle_3pts(a,b,c))
    feats = np.concatenate([feats, np.array(angles, dtype=np.float32)])
    return feats.astype(np.float32)

print("Loaded math utils.")


Loaded math utils.


In [6]:

# =======================================================
# 4) TRÍCH XUẤT KEYPOINTS TỪ VIDEO (hoặc webcam) -> .npz
# =======================================================
def extract_features_from_video(
    video_path: str,
    flip_horizontal: bool = True,
    save_npz_path: Optional[str] = None,
    use_world: bool = False,
    max_frames: Optional[int] = None,
) -> Dict[str, np.ndarray]:
    """Trích xuất 33 landmarks (2D) từ MediaPipe Pose, chuyển thành feature theo frame.
    Trả về dict: { 'features': (T,F), 'landmarks': (T,33,2), 'valid': (T,), 'fps': float }
    """
    if mp is None:
        raise RuntimeError("mediapipe chưa sẵn sàng. Hãy cài mediapipe trước.")
    mp_pose = mp.solutions.pose

    cap = cv2.VideoCapture(0 if video_path == 'webcam' else video_path)
    if not cap.isOpened():
        raise IOError(f"Không mở được nguồn video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    frames_feats, frames_lm, valid = [], [], []

    with mp_pose.Pose(static_image_mode=False, model_complexity=1,
                      enable_segmentation=False, smooth_landmarks=True) as pose:
        t = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if flip_horizontal:
                frame = cv2.flip(frame, 1)
            h, w = frame.shape[:2]
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            res = pose.process(rgb)

            if res.pose_landmarks:
                lm = np.array([(p.x*w, p.y*h) for p in res.pose_landmarks.landmark], dtype=np.float32)
                feat = landmarks_to_feature(lm)
                frames_feats.append(feat)
                frames_lm.append(lm)
                valid.append(1)
            else:
                # khung không nhận landmark
                if len(frames_feats) > 0:
                    frames_feats.append(frames_feats[-1])  # giữ last-known
                    frames_lm.append(frames_lm[-1])
                else:
                    frames_feats.append(np.zeros(66+len(ANGLE_TRIPLETS), dtype=np.float32))
                    frames_lm.append(np.zeros((MP_LANDMARK_NUM,2), dtype=np.float32))
                valid.append(0)

            t += 1
            if (max_frames is not None) and (t >= max_frames):
                break

    cap.release()

    feats = np.stack(frames_feats, axis=0) if frames_feats else np.zeros((0, 66+len(ANGLE_TRIPLETS)), dtype=np.float32)
    lms = np.stack(frames_lm, axis=0) if frames_lm else np.zeros((0, MP_LANDMARK_NUM, 2), dtype=np.float32)
    valid = np.array(valid, dtype=np.uint8)
    out = {'features': feats, 'landmarks': lms, 'valid': valid, 'fps': float(fps)}

    if save_npz_path is not None:
        np.savez_compressed(save_npz_path, **out)
        print("Đã lưu:", save_npz_path)
    return out

print("Hàm trích xuất đã sẵn sàng. Cấu trúc dataset: data_videos/<action>/*.mp4")


Hàm trích xuất đã sẵn sàng. Cấu trúc dataset: data_videos/<action>/*.mp4


In [7]:

# ===================================================
# 5) TIỀN XỬ LÝ & GỘP DỮ LIỆU TRAIN THEO ACTION
# ===================================================
def build_training_matrix(action: str) -> Tuple[np.ndarray, List[int], List[float]]:
    """Đọc toàn bộ .mp4 / .mov trong data_videos/<action>,
    trích xuất features và gộp thành ma trận (T,F). Trả về:
      X_all: (N,F) – nối các frames
      seg_ids: danh sách cumulative số frame theo video để có thể quy chiếu
      fps_list: fps từng video
    """
    act_dir = DATA_DIR / action
    act_dir.mkdir(parents=True, exist_ok=True)
    video_paths = [p for p in act_dir.glob("*.*") if p.suffix.lower() in [".mp4",".mov",".avi",".mkv"]]
    X_all = []
    seg_ids = []
    fps_list = []
    for vid in video_paths:
        npz_path = FEAT_DIR / f"{action}__{vid.stem}.npz"
        if not npz_path.exists():
            _ = extract_features_from_video(str(vid), flip_horizontal=True, save_npz_path=str(npz_path))
        data = np.load(npz_path, allow_pickle=True)
        feats = data["features"]
        X_all.append(feats)
        seg_ids.append(len(feats))
        fps_list.append(float(data["fps"]))
    if len(X_all) == 0:
        raise RuntimeError(f"Không có video cho action '{action}' trong {act_dir}. Hãy thêm video rồi chạy lại.")
    X_all = np.concatenate(X_all, axis=0)
    return X_all, seg_ids, fps_list

print("Hàm gộp dữ liệu train sẵn sàng.")


Hàm gộp dữ liệu train sẵn sàng.


In [8]:

# ===================================================
# 6) HUẤN LUYỆN PHA A→E CHO MỖI ACTION
# ===================================================
def train_action_phases(
    action: str,
    num_phases: int = NUM_PHASES,
    method: str = "auto"  # 'auto'|'hmm'|'kmeans'
) -> Dict:
    """Học pha A→E từ dữ liệu frame-level:
    - Nếu có hmmlearn: dùng HMM (GaussianHMM) -> states ~ phases
    - Nếu không: KMeans trên toàn bộ frames, sau đó sắp xếp cluster theo thời gian xuất hiện
    Trả về dict template gồm:
      {
        'action': action,
        'num_phases': num_phases,
        'scaler': scaler (mean/std),
        'centroids': [centroid_feat_i],
        'tolerances': [tol_i]  # MAD-based
      }
    """
    X_all, seg_ids, fps_list = build_training_matrix(action)
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X_all)

    # Tính chỉ số thời gian xuất hiện ưu thế để sắp thứ tự pha
    frame_indices = np.arange(len(Xs))

    if method == "auto":
        method = "hmm" if HAVE_HMM else "kmeans"

    if method == "hmm" and HAVE_HMM:
        # HMM Gaussian: states ~ phases
        hmm = GaussianHMM(n_components=num_phases, covariance_type='full', n_iter=100, verbose=False, random_state=42)
        hmm.fit(Xs)
        states = hmm.predict(Xs)
        # Thứ tự pha = thứ tự trung vị chỉ số frame của từng state
        order = []
        for s in range(num_phases):
            idx = np.where(states == s)[0]
            median_t = np.median(idx) if len(idx) else 1e9
            order.append((median_t, s))
        order.sort()
        ordered_states = [s for _, s in order]
        # Tính centroid theo state đã sắp
        centroids = []
        tolerances = []
        for s in ordered_states:
            idx = np.where(states == s)[0]
            if len(idx) == 0:
                centroids.append(np.zeros(Xs.shape[1], dtype=np.float32))
                tolerances.append(DEFAULT_TOL)
                continue
            c = Xs[idx].mean(axis=0)
            centroids.append(c.astype(np.float32))
            # MAD-based tol
            dist = np.linalg.norm(Xs[idx] - c, axis=1)
            mad = np.median(np.abs(dist - np.median(dist))) + 1e-6
            tolerances.append(float(2.5 * mad))  # hệ số có thể tinh chỉnh
    else:
        # KMeans fallback
        km = KMeans(n_clusters=num_phases, random_state=42, n_init=10)
        states = km.fit_predict(Xs)
        # Sắp xếp cluster theo thời điểm xuất hiện trung vị
        order = []
        for s in range(num_phases):
            idx = np.where(states == s)[0]
            median_t = np.median(idx) if len(idx) else 1e9
            order.append((median_t, s))
        order.sort()
        ordered_states = [s for _, s in order]
        # Tính centroid theo cluster đã sắp
        centroids = []
        tolerances = []
        for s in ordered_states:
            idx = np.where(states == s)[0]
            if len(idx) == 0:
                centroids.append(np.zeros(Xs.shape[1], dtype=np.float32))
                tolerances.append(DEFAULT_TOL)
                continue
            c = Xs[idx].mean(axis=0)
            centroids.append(c.astype(np.float32))
            dist = np.linalg.norm(Xs[idx] - c, axis=1)
            mad = np.median(np.abs(dist - np.median(dist))) + 1e-6
            tolerances.append(float(2.5 * mad))

    template = {
        "action": action,
        "num_phases": num_phases,
        "scaler_mean": scaler.mean_.tolist(),
        "scaler_scale": scaler.scale_.tolist(),
        "centroids": [c.tolist() for c in centroids],
        "tolerances": tolerances,
        "feature_dim": int(Xs.shape[1]),
        "angle_triplets": ANGLE_TRIPLETS,
        "created_at": time.time()
    }
    out_path = MODEL_DIR / f"{action}_template.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(template, f, ensure_ascii=False, indent=2)
    print(f"✅ Đã lưu template A→E cho action='{action}' vào {out_path}")
    print("Pha tương ứng: A=0, B=1, C=2, D=3, E=4")
    return template

print("Hàm huấn luyện pha đã sẵn sàng.")


Hàm huấn luyện pha đã sẵn sàng.


In [9]:

# ===================================================
# 7) ĐỌC TEMPLATE & HIỆU CHỈNH (CALIBRATION)
# ===================================================
def load_action_template(action: str) -> Dict:
    path = MODEL_DIR / f"{action}_template.json"
    if not path.exists():
        raise FileNotFoundError(f"Chưa có template cho action '{action}'. Hãy train trước (Cell 6).")
    with open(path, "r", encoding="utf-8") as f:
        tpl = json.load(f)
    return tpl

def make_scaler_from_template(tpl: Dict):
    mean = np.array(tpl["scaler_mean"], dtype=np.float32)
    scale = np.array(tpl["scaler_scale"], dtype=np.float32)
    def transform(X):
        return (X - mean) / (scale + 1e-8)
    return transform

def calibrate_user_from_frames(frames_feats: np.ndarray, tpl: Dict) -> Dict:
    """Hiệu chỉnh dung sai theo người dùng: đo khoảng cách đến từng centroid
    trong một đoạn khởi động ngắn -> điều chỉnh tolerance (nới/siết) hợp lý.
    """
    transform = make_scaler_from_template(tpl)
    Xs = transform(frames_feats)
    centroids = [np.array(c, dtype=np.float32) for c in tpl["centroids"]]
    # Tính khoảng cách tối thiểu đến mỗi centroid
    dists = []
    for i, c in enumerate(centroids):
        dist_i = np.linalg.norm(Xs - c[None,:], axis=1)
        dists.append(np.median(dist_i))
    # Tạo scale factor dựa trên median khoảng cách
    med = np.median(dists)
    cal_factor = float(np.clip(med, 0.5, 2.0))  # giới hạn
    adj_tols = [float(t * cal_factor) for t in tpl["tolerances"]]
    return {
        "adj_tolerances": adj_tols,
        "cal_factor": cal_factor
    }

print("Hàm đọc template & hiệu chỉnh đã sẵn sàng.")


Hàm đọc template & hiệu chỉnh đã sẵn sàng.


In [10]:

# ===================================================
# 8) ĐÁNH GIÁ KHUNG XƯƠNG ĐÚNG/SAI THEO PHA
# ===================================================
def phase_similarity(feat: np.ndarray, tpl: Dict, phase_idx: int) -> float:
    transform = make_scaler_from_template(tpl)
    X = transform(feat)
    c = np.array(tpl["centroids"][phase_idx], dtype=np.float32)
    return float(np.linalg.norm(X - c))

def joints_correct_mask(
    lm: np.ndarray,
    tpl_lm_ref: Optional[np.ndarray],
    tol: float = 0.12
) -> np.ndarray:
    """So sánh từng khớp với template (dạng landmark 2D đã chuẩn hoá),
    trả về mask đúng/sai cho 33 joints.
    """
    if tpl_lm_ref is None:
        return np.ones((MP_LANDMARK_NUM,), dtype=np.uint8)
    lm_norm = normalize_landmarks(lm)
    d = np.linalg.norm(lm_norm - tpl_lm_ref, axis=1)  # per-joint
    return (d < tol).astype(np.uint8)

def centroid_to_landmarks(centroid_feat: np.ndarray) -> np.ndarray:
    """Chuyển một centroid feature (66 + angles) về landmarks 33x2 (chuẩn hoá)."""
    coords = centroid_feat[:66].reshape(MP_LANDMARK_NUM, 2)
    return coords

print("Hàm đánh giá khung xương đã sẵn sàng.")


Hàm đánh giá khung xương đã sẵn sàng.


In [ ]:

# ===================================================
# 9) REAL-TIME SHADOWING UI (ABC... + SKELETON)
# ===================================================
class ShadowingUI:
    def __init__(self, window_name="Shadowing", scale=1.0, fullscreen=False):
        self.window = window_name
        self.scale = scale
        self.fullscreen = fullscreen
        self.click_points = []
        cv2.namedWindow(self.window, cv2.WINDOW_NORMAL)
        if self.fullscreen:
            cv2.setWindowProperty(self.window, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.setMouseCallback(self.window, self.on_mouse)

    def on_mouse(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.click_points.append((x,y))

    def draw_skeleton(self, frame, lm, correct_mask):
        for (i,j) in SKELETON_EDGES:
            p1 = tuple(lm[i].astype(int))
            p2 = tuple(lm[j].astype(int))
            ok = bool(correct_mask[i] and correct_mask[j])
            color = (0,255,0) if ok else (0,0,255)
            cv2.line(frame, p1, p2, color, 2)
        # joints
        for idx, p in enumerate(lm):
            ok = bool(correct_mask[idx])
            color = (0,255,0) if ok else (0,0,255)
            cv2.circle(frame, tuple(p.astype(int)), 3, color, -1)

    def draw_phase_points(self, frame, phase_points: List[Tuple[int,int]], active_idx: int):
        # Vẽ các điểm ABCDE to, dễ bấm
        for k, (x,y) in enumerate(phase_points):
            color = (0,200,255) if k == active_idx else (200,200,200)
            cv2.circle(frame, (x,y), 18, color, -1)
            cv2.putText(frame, chr(ord('A')+k), (x-7, y+6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (20,20,20), 2)

    def present(self, frame):
        if self.scale != 1.0:
            h,w = frame.shape[:2]
            frame = cv2.resize(frame, (int(w*self.scale), int(h*self.scale)))
        cv2.imshow(self.window, frame)

    def handle_keys(self) -> bool:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            return False
        elif key == ord('+') or key == ord('='):
            self.scale = min(3.0, self.scale + 0.1)
        elif key == ord('-') or key == ord('_'):
            self.scale = max(0.5, self.scale - 0.1)
        elif key == ord('f'):
            self.fullscreen = not self.fullscreen
            cv2.setWindowProperty(self.window, cv2.WND_PROP_FULLSCREEN,
                                  cv2.WINDOW_FULLSCREEN if self.fullscreen else cv2.WINDOW_NORMAL)
        return True

def run_shadowing(action: str = "serve", flip_horizontal: bool = True):
    if mp is None:
        raise RuntimeError("mediapipe chưa được cài. Không thể chạy real-time.")
    tpl = load_action_template(action)
    transform = make_scaler_from_template(tpl)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Không mở được webcam.")
    ui = ShadowingUI(window_name=f"Shadowing - {action}", scale=1.2, fullscreen=False)
    mp_pose = mp.solutions.pose

    curr_phase = 0  # A=0,...,E=4
    achieved = [False]*tpl["num_phases"]
    start_time, end_time = None, None
    phase_points = None  # toạ độ lớn để người dùng bấm theo trình tự

    with mp_pose.Pose(static_image_mode=False, model_complexity=1,
                      enable_segmentation=False, smooth_landmarks=True) as pose:
        t0 = time.time()
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if flip_horizontal:
                frame = cv2.flip(frame, 1)
            h,w = frame.shape[:2]

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            res = pose.process(rgb)

            if res.pose_landmarks:
                lm = np.array([(p.x*w, p.y*h) for p in res.pose_landmarks.landmark], dtype=np.float32)
                feat = landmarks_to_feature(lm)
                # so khớp với phase hiện tại
                dist = phase_similarity(feat, tpl, curr_phase)
                tol = tpl["tolerances"][curr_phase]
                # khung xương đúng/sai per-joint
                tpl_lm = centroid_to_landmarks(np.array(tpl["centroids"][curr_phase], dtype=np.float32))
                mask = joints_correct_mask(lm, tpl_lm, tol=0.12)
                # vẽ skeleton
                ui.draw_skeleton(frame, lm, mask)

                # Tạo điểm ABCDE lớn trên màn hình (gần vị trí khớp quan trọng – ví dụ cổ tay phải theo template)
                if phase_points is None:
                    # Lấy 1 khớp đại diện (vd: cổ tay phải 16, khuỷu 14, vai 12) từ template và chiếu tỉ lệ lên khung hình
                    tpl_lm_norm = tpl_lm  # đã chuẩn hoá
                    # Chiếu đơn giản: lấy vai phải (12) làm gốc gần giữa ảnh
                    base = np.array([w*0.7, h*0.3])
                    # Đặt 5 điểm theo cột dọc
                    phase_points = [(int(base[0]), int(base[1] + i*60)) for i in range(tpl["num_phases"])]

                # Kiểm tra click vào đúng điểm kế tiếp
                if ui.click_points:
                    cx, cy = ui.click_points.pop(0)
                    px, py = phase_points[curr_phase]
                    if (cx - px)**2 + (cy - py)**2 < 28**2:
                        # click trúng điểm của phase hiện tại -> xác nhận
                        achieved[curr_phase] = True

                # Nếu khoảng cách < tol -> coi như đạt pha
                if dist < tol:
                    achieved[curr_phase] = True

                # Khi đạt A lần đầu -> đánh dấu start_time
                if achieved[0] and start_time is None:
                    start_time = time.time()

                # Nếu đã đạt phase hiện tại -> chuyển tiếp
                if achieved[curr_phase] and curr_phase < tpl["num_phases"] - 1:
                    curr_phase += 1
                elif achieved[-1]:
                    end_time = time.time()
                    cv2.putText(frame, "Sequence COMPLETED!", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)

                # Vẽ điểm ABCDE
                ui.draw_phase_points(frame, phase_points, curr_phase)

                # Overlay thông tin
                cv2.putText(frame, f"Phase: {chr(ord('A')+curr_phase)}  dist={dist:.3f} tol={tol:.3f}",
                            (20, h-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
            else:
                cv2.putText(frame, "No pose detected", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2)

            ui.present(frame)
            if not ui.handle_keys():
                break

    cap.release()
    cv2.destroyAllWindows()

    if start_time and end_time:
        result = {
            "action": action,
            "start_time_unix": start_time,
            "end_time_unix": end_time,
            "duration_sec": end_time - start_time
        }
        out_json = MODEL_DIR / f"last_{action}_segment.json"
        with open(out_json, "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
        print("Đã lưu thời điểm bắt đầu/kết thúc:", out_json)
    else:
        print("Chưa hoàn tất trọn vẹn chuỗi A→E.")


In [ ]:

# ===================================================
# 10) PHÂN ĐOẠN VIDEO MỚI -> JSON (BẮT ĐẦU / KẾT THÚC)
# ===================================================
def segment_video_by_template(video_path: str, action: str, flip_horizontal=True) -> Dict:
    tpl = load_action_template(action)
    data = extract_features_from_video(video_path, flip_horizontal=flip_horizontal, save_npz_path=None)
    feats, valid, fps = data["features"], data["valid"], float(data["fps"])
    transform = make_scaler_from_template(tpl)

    curr_phase = 0
    in_sequence = False
    start_idx, end_idx = None, None

    for i in range(len(feats)):
        f = feats[i]
        dist = phase_similarity(f, tpl, curr_phase)
        tol = tpl["tolerances"][curr_phase]
        if dist < tol:
            if curr_phase == 0 and not in_sequence:
                in_sequence = True
                start_idx = i
            if curr_phase < tpl["num_phases"]-1:
                curr_phase += 1
            else:
                end_idx = i
                break

    result = {
        "action": action,
        "video": video_path,
        "start_frame": int(start_idx) if start_idx is not None else None,
        "end_frame": int(end_idx) if end_idx is not None else None,
        "fps": fps,
        "start_time_sec": (start_idx / fps) if start_idx is not None else None,
        "end_time_sec": (end_idx / fps) if end_idx is not None else None
    }
    out_json = MODEL_DIR / f"segment_{Path(video_path).stem}_{action}.json"
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    print("Kết quả phân đoạn lưu tại:", out_json)
    return result

print("Hàm phân đoạn video đã sẵn sàng.")


Hàm phân đoạn video đã sẵn sàng.



## 🚀 Hướng dẫn chạy nhanh

1. **Chuẩn bị dữ liệu huấn luyện**  
   - Thả video vào thư mục `data_videos/<action>/`. Ví dụ: `data_videos/serve/serve_01.mp4`.
   - Sửa danh sách `ACTIONS` ở **Cell 2** nếu có thêm động tác.

2. **Trích xuất & Huấn luyện**  
   - Chạy **Cell 5** để gộp dữ liệu.  
   - Chạy **Cell 6**: `train_action_phases("serve")` để tạo template A→E.

3. **Hiệu chỉnh dung sai theo người dùng (tuỳ chọn)**  
   - Quay một đoạn ngắn bằng webcam: chạy **Cell 4** với `video_path="webcam", max_frames=300` để lấy `frames_feats`.  
   - Gọi **Cell 7**: `calibrate_user_from_frames(frames_feats, tpl)` để điều chỉnh `tolerances`.

4. **Chạy real-time shadowing**  
   - Chạy **Cell 9**: `run_shadowing("serve")`.  
   - Phím tắt: **q** thoát, **+/-** phóng to/thu nhỏ, **f** toàn màn hình.  
   - **Click vào các điểm A–E** theo trình tự hoặc đơn giản là thực hiện động tác đúng để tự qua pha.  
   - Khung xương: **xanh** = đúng, **đỏ** = sai.

5. **Phân đoạn video offline**  
   - Dùng **Cell 10**: `segment_video_by_template("test.mp4", "serve")` để nhận `start_time_sec`/`end_time_sec` dạng JSON.

> 🧩 Bạn có thể thay **KMeans** bằng **HMM** (nếu cài `hmmlearn`) chỉ bằng tham số `method="hmm"` trong `train_action_phases`.



## 🧠 Mẹo & Mở rộng
- **Độ chính xác cao hơn**: tăng chất lượng video (ánh sáng), dùng `model_complexity=2` của MediaPipe, tăng số pha >5 đối với động tác phức tạp.
- **Tô màu theo khớp quan trọng**: cân nặng hoá các joints (vd: vai–khuỷu–cổ tay khi *serve*).
- **Điểm A–E theo hình học thực**: hiện tại demo đặt A–E ở cột phải. Bạn có thể nội suy từ centroid template -> chiếu lên ảnh theo scale/offset để sát tư thế hơn.
- **Lưu vết luyện tập**: ghi log độ lệch từng pha, số lần thử, thời gian hoàn thành để feedback tiến bộ.
- **Thiết bị cảm ứng**: OpenCV hỗ trợ **click chuột**; nếu màn hình cảm ứng map thành sự kiện chuột, vẫn hoạt động.
